<a href="https://colab.research.google.com/github/ayushmangupta/TF2/blob/master/TF2_MobileNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
%load_ext tensorboard
#!pip3 install tensorflow-gpu==2.0.0-beta1
import os
import datetime
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.keras as tfk
import numpy as np
import tensorflow.keras.layers as tfkl
import tensorflow_datasets as tfds

tfds.disable_progress_bar()
print(tf.__version__)
log_dir = "logd"
!rm -rf logd/

2.0.0-beta1


In [0]:
def plot_filters(model,ii):
    d = model.layers[0]
    o = d(ii)
    mainImg=[]
    for i in range(8): 
        a = []
        for j in range(8):
            a.append(o[i,:,:,j])
        imgg = np.hstack(a)
        mainImg.append(imgg)
    return np.vstack(mainImg)

In [0]:
#img = tf.random.normal([10,28,28,3])

In [0]:
def depth_conv_block(strides,alpha,filters): 
        layer = tfk.Sequential([
        tfkl.DepthwiseConv2D(1,strides=strides,padding="same"),
        tfkl.BatchNormalization(),
        tfkl.Activation('relu'),
        tfkl.Conv2D(int(filters*alpha),(1,1),padding="same"),
        tfkl.BatchNormalization(),
        tfkl.Activation('relu') ])
        return layer
    
class Model(tfk.Model):
    def __init__(self):
        super(Model,self).__init__()
        self.input_layer1 = tfkl.InputLayer(input_shape=(28,28,1))
        self.padding = tfkl.ZeroPadding2D(padding=(2,2))
        self.conv1 = tfkl.Conv2D(32,kernel_size=3,padding="same")
        self.batchnorm = tfkl.BatchNormalization()
        self.activation = tfkl.Activation('relu')
        self.l1 = depth_conv_block(strides=(1,1),alpha=1,filters=64)
        self.l2 = depth_conv_block(strides=(2,2),alpha=1,filters=128)
        self.l3 = depth_conv_block(strides=(1,1),alpha=1,filters=128)
        self.l4 = depth_conv_block(strides=(2,2),alpha=1,filters=256)
        self.l5 = depth_conv_block(strides=(1,1),alpha=1,filters=256)
        self.l6 = depth_conv_block(strides=(2,2),alpha=1,filters=512)
        self.l7 = depth_conv_block(strides=(1,1),alpha=1,filters=512)
        self.l8 = depth_conv_block(strides=(2,2),alpha=1,filters=1024)
        self.l9 = depth_conv_block(strides=(1,1),alpha=1,filters=1024)
        self.flatten = tfkl.Flatten()
        self.globalAvg  = tfkl.GlobalAveragePooling2D()
        self.d1 = tfkl.Dense(10,activation="softmax")

    
    def call(self,x):
        x = self.input_layer1(x)
        x = self.padding(x)
        #print("after padding",x.shape)
        x = self.conv1(x)
        x = self.batchnorm(x)
        x = self.activation(x)
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        x = self.l4(x)
        x = self.l5(x)
        x = self.l6(x)
        for _ in range(5):
            x = self.l7(x)
        x = self.l8(x)
        x = self.l9(x)
        #print("after l9",x.shape)
        x = self.globalAvg(x)
        #print("after globalAvg",x.shape)
        x = self.flatten(x)
        x = self.d1(x)
        return x
        

In [0]:
mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = tf.cast(x_train,tf.float32)
x_test = tf.cast(x_test,tf.float32)


# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.RMSprop(0.01)
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')


In [0]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)


In [0]:
model = Model()
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
test_log_dir = 'logd/gradient_tape/' + current_time + '/test'

In [0]:
test_summary_writer = tf.summary.create_file_writer(test_log_dir)
ii = x_train[:10,:,:,:]

In [0]:
EPOCHS = 15
#tf.summary.trace_on(graph=True, profiler=True)
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(images, labels)
    

    for test_images, test_labels in test_ds:
      test_step(test_images, test_labels)

    with test_summary_writer.as_default():
        tf.summary.scalar('loss', test_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
        #filter_img = plot_filters(model,ii)
        #filter_img = tf.summary.image('image',filter_img[np.newaxis,:,:,np.newaxis],step=epoch)
        #tf.summary.trace_export(name="my_func_trace", step=0, profiler_outdir=train_log_dir)
        
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print (template.format(epoch+1,
                           train_loss.result(),
                           train_accuracy.result()*100,
                           test_loss.result(),
                           test_accuracy.result()*100))
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 2.3057780265808105, Accuracy: 9.891666412353516, Test Loss: 2.3031139373779297, Test Accuracy: 10.0
Epoch 2, Loss: 2.304877519607544, Accuracy: 9.904999732971191, Test Loss: 2.3031139373779297, Test Accuracy: 10.0
Epoch 3, Loss: 2.304574728012085, Accuracy: 9.909443855285645, Test Loss: 2.3031139373779297, Test Accuracy: 10.0
Epoch 4, Loss: 2.3044190406799316, Accuracy: 9.911666870117188, Test Loss: 2.3031139373779297, Test Accuracy: 10.0
